In [ ]:
!rm -fr oscillate
!git clone https://github.com/abreham-atlaw/oscillate/
!mkdir out
!pip install bpemb
!pip install encodec

Cloning into 'oscillate'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 215 (delta 74), reused 203 (delta 62), pack-reused 0
Receiving objects: 100% (215/215), 42.26 KiB | 5.28 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf  LJSpeech-1.1.tar.bz2

In [ ]:
import sys
sys.path.append("/kaggle/working/oscillate/src/python")

In [ ]:
import pandas as pd

import os

from oscillate.data.prepare.data_preparer import DataPreparer
from oscillate.data.prepare.encoders.audio import EncodecEncoder
from oscillate.data.prepare.encoders.text.bpemb_encoder import BpembEncoder

In [ ]:
NOTEBOOK_INDEX = 0

CHECKPOINT = 2000
NOTEBOOK_SIZE = 100

START_INDEX = NOTEBOOK_INDEX * NOTEBOOK_SIZE
END_INDEX = START_INDEX + NOTEBOOK_SIZE

ENCODEC_BANDWIDTH = 12
ENCODEC_EOS_TOKEN = 0

BPEMB_LANG = "en"
BPEMB_EMB_SIZE = 50

TEXT_PADDING_TOKEN = 0
TEXT_BLOCK_SIZE = 256

AUDIO_PADDING_TOKEN = 0
AUDIO_BLOCK_SIZE = 512

AUDIO_FILE_FORMAT = "{}.wav"
AUDIO_DIR = os.path.abspath("/kaggle/working/LJSpeech-1.1/wavs")
CSV_PATH = os.path.abspath("/kaggle/working/LJSpeech-1.1/metadata.csv")
SAVE_PATH = os.path.abspath("/kaggle/working/out")

CSV_HEADER_TEXT = "text_normalized"
CSV_HEADER_AUDIO = "audio"

TEST_SPLIT_SIZE = 0.3



In [ ]:
audio_encoder = EncodecEncoder(bandwidth=ENCODEC_BANDWIDTH, eos_token=ENCODEC_EOS_TOKEN)

In [ ]:
text_encoder = BpembEncoder(lang=BPEMB_LANG, emb_size=BPEMB_EMB_SIZE)

In [ ]:
preparer = DataPreparer(
    audio_encoder,
    text_encoder,
    audio_block_size=AUDIO_BLOCK_SIZE,
    text_block_size=TEXT_BLOCK_SIZE,
    audio_dir=AUDIO_DIR,
    audio_file_format=AUDIO_FILE_FORMAT,
    checkpoint=CHECKPOINT
)

In [ ]:
df = pd.read_csv(CSV_PATH, sep="|")
df.columns = [CSV_HEADER_AUDIO, "text", CSV_HEADER_TEXT]
df = df.iloc[START_INDEX:END_INDEX]

In [ ]:
preparer.start(
    df=df,
    save_path=SAVE_PATH,
    header_text=CSV_HEADER_TEXT,
    header_audio=CSV_HEADER_AUDIO
)

In [ ]:
!rm -fr LJSpeech-1.1 LJSpeech-1.1.tar.bz2 oscillate